# Testing Instagram's API
https://github.com/LevPasha/Instagram-API-python

## Login

In [ ]:
from time import sleep
import random

In [ ]:
from InstagramAPI import InstagramAPI # check link above for installation instructions

In [ ]:
api = InstagramAPI("your_username", "your_password")
api.login()

## User ID and followers/following

In [ ]:
user_id = api.username_id

In [ ]:
followers = api.getTotalFollowers(user_id) #dictionary with information about followers
followers_user_id = [dic["pk"] for dic in followers] #list with followers' ids

In [ ]:
following = api.getTotalFollowings(user_id) #dictionary with information about following
following_user_id = [dic["pk"] for dic in following] # list with followings' ids

## New Followings

In [ ]:
# Get users that posted media with certain hashtag
def follow_from_hashtag(hashtag):
    api.getHashtagFeed(hashtag)
    hashtag_feed = api.LastJson
    return([dic["user"]["pk"] for dic in hashtag_feed["items"]])

# Get media ids for the last 4 medias poted by a certain user
def like_from_new_following(user_id):
    api.getUserFeed(user_id)
    user_feed = api.LastJson
    return([dic["id"] for dic in user_feed["items"][0:4]])

In [ ]:
# find users that posted media with certain hashtags, like their last 4 posts and then follow them
def new_followings(hashtags):
    users_to_follow = []
    for hashtag in hashtags:
        users_to_follow.extend(follow_from_hashtag(hashtag))
    users_to_follow = set(users_to_follow)
    users_to_follow.difference(set(following_user_id))
    users_to_follow = list(random.sample(users_to_follow, 300)) # be sure to check the limits so IG doesn't block your account. I'm playing safe with 300
    for user_id in users_to_follow:
        medias_to_like = like_from_new_following(user_id)
        for media_id in medias_to_like:
            api.like(media_id)
            sleep(2)
        api.follow(user_id)
        sleep(random.randint(40,60)) # also check the limit for requests per second

In [ ]:
new_followings(["zeedog", "bestwoof", "bordercolliebrasil", "zeedogmafia", "cachorrosdobrasil", "bordercolliesrock"])

## Unfollow Unfollowers

In [ ]:
# unfollow people who are not following you
def unfollow_unfollowers(followers_id, following_id, exceptions):
    unfollowers = list(set(following_id).difference(set(followers_id)))
    unfollow = [user_id for user_id in unfollowers if user_id not in exceptions]
    return(unfollow)

In [ ]:
# But don't unfollow these exceptions
username_exceptions = ["zakgeorge", "thesupercollies", "my_aussie_gal", "thesupercolliesmom", "zee_dog",
                      "thesupershepherds", "zeedogmafia", "zeedog", "buddyboy_bc", "ze.border", "kikopup"]
exceptions = [dic["pk"] for dic in following if dic["username"] in username_exceptions]

In [ ]:
unfollow = unfollow_unfollowers(followers_user_id, following_user_id, exceptions)
for user_id in unfollow:
    api.unfollow(user_id)
    sleep(random.randint(40,60))

## Send Love to Followers

In [ ]:
# Like the last 4 posts of certain user
def send_love_to_follower(follower_id):
    api.getUserFeed(follower_id)
    feed = api.LastJson
    medias_to_like = [dic["id"] for dic in feed["items"][0:4]]
    for media in medias_to_like:
        api.like(media)
        sleep(2)

In [ ]:
# select some followers too have their media liked. Don't worry, if you've already liked them, it won't "unilike" any.
def send_love_to_followers(followers):
    followers_id = random.sample([dic["pk"] for dic in followers if dic["is_private"] == False], 200)
    for follower in followers_id:
        send_love_to_follower(follower)
        sleep(random.randint(10,30))

In [ ]:
send_love_to_followers(followers)

## Logout

In [ ]:
api.logout()